### **Chatbot Using DialoGPT or RAG**
**two modes in that project:**
 *  DialoGPT: Pretrained conversational model (open-domain chatting)
 *  RAG (Retrieval-Augmented Generation): Answers grounded in factual documents
 (e.g., Wikipedia, knowledge base)

**model 1:** DialoGPT Chatbot (Open-Domain Conversation)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
# Load small DialoGPT model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
chat_history_ids = None
print(" Chatbot is ready! Type 'quit' to exit.")

for step in range(5):  # Limit the conversation to 5 turns
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break

    # Encode user input and add end-of-sentence token
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append new input to chat history
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    # Generate a reply from the model
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and display the latest response
    reply = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print("Bot:", reply)

 Chatbot is ready! Type 'quit' to exit.
You: hi
Bot: Hi
You: are you a bot?
Bot: hi bot
You: no you are
Bot: no you are
You: quit


## 📝 **Summary: DialoGPT-small Chatbot**

This project uses the `DialoGPT-small` model to create a simple conversational AI. It takes user input, maintains a short history, and generates replies based on context.

Since it's the smallest version of DialoGPT, it may repeat or mirror user input, especially in short or ambiguous conversations. It’s best for basic experimentation and understanding how transformer-based chat models work out-of-the-box.

✅ Lightweight, easy to run, and useful for learning chatbot fundamentals.


**model 2:RAG-Style Chatbot**


**Install Required Libraries**

In [1]:
!pip install transformers sentence-transformers

**Prepare a Mini Knowledge Base**

In [2]:
from sentence_transformers import SentenceTransformer, util

# Knowledge base (can be FAQs, wiki docs, etc.)
documents = [
    "George Washington was the first president of the United States.",
    "Barack Obama served as the 44th U.S. President.",
    "The capital of the United States is Washington, D.C.",
    "The United Nations was founded after World War II."
]

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(documents, convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Define the User Question and Retrieve Relevant Info**

In [3]:
# User question
query = "Who was the first U.S. president?"
query_embedding = embedder.encode(query, convert_to_tensor=True)

# Perform semantic search
hits = util.semantic_search(query_embedding, doc_embeddings, top_k=1)[0]
retrieved_text = documents[hits[0]['corpus_id']]

print("Retrieved Context:", retrieved_text)

Retrieved Context: George Washington was the first president of the United States.


**Generate an Answer with GPT-2**

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load text generation model (GPT-2)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Prepare prompt with retrieved context
prompt = f"Context: {retrieved_text}\nQuestion: {query}\nAnswer:"

# Generate response
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output_ids = model.generate(input_ids, max_length=100, do_sample=True, temperature=0.7)
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("\nBot:", response.split("Answer:")[-1].strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Bot: George Washington [U.S. President].
Question: Who


# 📝 Summary: RAG-Style Chatbot in Colab

This project simulates the functionality of Facebook’s RAG model using tools that fully work in Google Colab.

### 🔹 Components:
- **SentenceTransformer** (`all-MiniLM-L6-v2`)  
  → Used to embed and semantically search a set of documents (your knowledge base).
  
- **GPT-2**  
  → Used to generate natural language answers based on the top retrieved document.

### 🔹 Workflow:
1. Embed your document corpus using `SentenceTransformer`.
2. Encode the user’s question and find the most relevant document.
3. Feed the retrieved document + question as a prompt to GPT-2.
4. Generate an answer in context, similar to how RAG operates.

### ✅ Benefits:
- Fully Colab-compatible (no FAISS or retriever issues)
- Easy to modify (custom documents, models, prompt formats)
- Mimics retrieval + generation like `facebook/rag-token-base`